Derin Pekiştirmeli Ogrenme / Deep q Learning

In [1]:
# %pip install gym
# %pip install pygame

import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
from tqdm import tqdm


In [4]:
class DQLAgent:
    def __init__(self, env):
        #cevrenin gözlem alanı (state) boyutu
        self.state_size = env.observation_space.shape[0]

        #cevrede bulunan eylem sayisi (ajanın secebilcegi eylem sayisi)
        self.action_size = env.action_space.n

        #gelecekteki odullerin indirim orani
        self.gamma = 0.95

        self.learning_rate = 0.001

        self.epsilon = 1.0

        self.epsilon_decay = 0.995

        self.epsilon_min = 0.01

        self.memory = deque(maxlen=1000)

        self.model = self.build_model() # ANN

    def build_model(self):
        model = Sequential()

        model.add(Dense(48, input_dim = self.state_size, activation="relu"))
        model.add(Dense(24, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss = "mse", optimizer = Adam(learning_rate = self.learning_rate))

        return model

    def remember(self,state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self,state):
        
        if random.uniform(0,1) <= self.epsilon:
            return env.action_space.sample() # rastgle eylem seç
        
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])
    def replay(self,batch_size):
        
        if len(self.memory) < batch_size:
            return
        
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            if done : # eger done ise bitis durum var ise odulu dogrudan hedef olarak aliriz
                target = reward 
            else:
                target = reward + self.gamma*np.amax(self.model.predict(next_state, verbose=0)[0])

            # modelin tahmin ettigi oduller   
            train_target = self.model.predict(state, verbose=0)

            #ajanın yaptıgı eyleme gore tahmin edilen odulu guncelle
            train_target[0][action] = target

            #modeli egit 
            self.model.fit(state, train_target, verbose =0)

    def adaptiveEGreedy(self):

        self.epsilon > self.epsilon_min 
        self.epsilon = self.epsilon * self.epsilon_decay

# %% env kullanarak dql ajanı baslatma
env = gym.make("CartPole-v1", render_mode = "human") # cartpole ortamı baslatma
agent = DQLAgent(env)

batch_size = 32 # egitim için minibatch boyutu
episodes = 2 # epochs, simulasyonun oynatılacagı toplam bolum sayisi

for e in tqdm(range(episodes)):

    # ortamı sıfırla baslangıc durumunu al 
    state = env.reset()[0] # ortamı sıfırlamak 
    state = np.reshape(state, [ 1,4])
    time = 0 # zamanı adimi baslat
    
    while True:
        # ajan eylem secer 
        action = agent.act(state)

        # ajanımız ortamda bu eylemi uygular ve bu eylem sonucunda next_state,reward , bitis bilgisi(done) alir
        (next_state, reward, done , _ , _ ) = env.step(action)
        next_state = np.reshape(state,[1,4])

        #yapmıs oldugu bu adimi yani eylemi ve bu eylem sonucu env alinan bilgileri kaydeder 
        agent.remember(state,action,reward,next_state,done)

        # mevct durumu gğnceller
        state = next_state

        # deneyimleerden yeniden oynatmayı baslatır reply() => tarning
        agent.replay(batch_size)

        # epsilonu set eder 
        agent.adaptiveEGreedy()

        # zaman adimini arttırır
        time = time + 1

        # eger done ise donguyu kirar ve bolum biter ve yeni bolume baslar 
        if done: 
            print(f"Episode: {e}, time : {time}")
            break

# test edilmesi 
import time 

trained_model = agent 
env = gym.make("CartPole-v1", render_mode = "human") # cartpole ortamı baslatma

state = env.reset()[0]
state = np.reshape(state, [1,4])

time_t = 0 

while True:
    env.render()
    action = trained_model.act(state)
    (next_state, reward, done , _ , _ ) = env.step(action)
    next_state = np.reshape(next_state, [1,4])
    state = next_state 
    time_t += 1 
    print(f"time : {time_t}")

    time.sleep(0.5)

    if done:
        break 
        print ("Done")



 50%|█████     | 1/2 [00:00<00:00,  3.64it/s]

Episode: 0, time : 12


100%|██████████| 2/2 [00:00<00:00,  3.41it/s]

Episode: 1, time : 10
time : 1


time : 2
time : 3
time : 4
time : 5
time : 6
time : 7
time : 8
time : 9
time : 10
time : 11
time : 12
time : 13
time : 14
time : 15
time : 16
time : 17
time : 18
time : 19
time : 20
time : 21
time : 22
time : 23
time : 24
time : 25
time : 26
time : 27
time : 28
time : 29
time : 30
time : 31
time : 32
time : 33
time : 34
time : 35
time : 36
time : 37
time : 38
time : 39
time : 40
time : 41
time : 42
time : 43
time : 44
time : 45
time : 46
time : 47
time : 48
time : 49
time : 50
time : 51
time : 52
time : 53
time : 54
